# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

## Environment Setup

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Create our QandA application

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings

In [4]:
file = 'appendix_data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [5]:
embeddings = OpenAIEmbeddings()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/Users/gyuuakane/opt/anaconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/gyuuakane/opt/anaconda3/envs/langchain/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [6]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

/Users/gyuuakane/opt/anaconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### Coming up with test datapoints

In [7]:
data[10]

Document(metadata={'source': 'appendix_data/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [8]:
data[11]

Document(metadata={'source': 'appendix_data/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

### Hard-coded examples

In [9]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [10]:
from langchain.evaluation.qa import QAGenerateChain


In [11]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [12]:
# the warning below can be safely ignored

In [13]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/Users/gyuuakane/opt/anaconda3/envs/langchain/lib/python3.9/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [14]:
new_examples[0]

{'qa_pairs': {'query': "What is the approximate weight of a pair of Women's Campside Oxfords?",
  'answer': "The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz."}}

In [15]:
data[0]

Document(metadata={'source': 'appendix_data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

### Combine examples

In [16]:
examples += new_examples

In [17]:
qa.run(examples[0]["query"])

/Users/gyuuakane/opt/anaconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set does have side pockets.'

## Manual Evaluation

In [18]:
import langchain
langchain.debug = True

In [19]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

'Yes, the Cozy Comfort Pullover Set does have side pockets.'

In [20]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [21]:
for i, ex in enumerate(examples):
    print(f"Example {i}:")
    print(ex)
    print()

Example 0:
{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}

Example 1:
{'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection'}

Example 2:
{'qa_pairs': {'query': "What is the approximate weight of a pair of Women's Campside Oxfords?", 'answer': "The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz."}}

Example 3:
{'qa_pairs': {'query': 'What are some of the key features of the Recycled Waterhog Dog Mat, Chevron Weave?', 'answer': 'Some key features of the Recycled Waterhog Dog Mat, Chevron Weave include being constructed from recycled plastic materials, having thick and thin fibers for scraping dirt and absorbing water, drying quickly, resisting fading, rotting, mildew, and shedding, and being suitable for use indoors or out.'}}

Example 4:
{'qa_pairs': {'query': "What are some key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-P

In [22]:
def get_query(example):
    if "query" in example:
        return example["query"]
    elif "qa_pairs" in example and "query" in example["qa_pairs"]:
        return example["qa_pairs"]["query"]
    else:
        raise KeyError("No query found in the example")

try:
    predictions = qa.batch([{"query": get_query(ex)} for ex in examples])
except KeyError as e:
    print(f"Error: {e}")
    print("Some examples do not contain a query. Please check your data.")



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [23]:
from langchain.evaluation.qa import QAEvalChain

In [24]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [28]:
# get the answer from the example
def get_answer(example):
    if "answer" in example:
        return example["answer"]
    elif "qa_pairs" in example and "answer" in example["qa_pairs"]:
        return example["qa_pairs"]["answer"]
    else:
        raise KeyError("No answer found in the example")

# prepare evaluation data
eval_data = [
    {
        "query": get_query(ex),
        "answer": get_answer(ex),
        "result": pred["result"]
    }
    for ex, pred in zip(examples, predictions)
]

# prepare prediction data
pred_data = [
    {
        "query": get_query(ex),
        "result": pred["result"]
    }
    for ex, pred in zip(examples, predictions)
]

graded_outputs = eval_chain.evaluate(eval_data, pred_data, question_key="query", prediction_key="result")

# print the results
for i, (ex, pred, grade) in enumerate(zip(examples, predictions, graded_outputs)):
    print(f"Example {i}:")
    print("Question: " + get_query(ex))
    print("Real Answer: " + get_answer(ex))
    print("Predicted Answer: " + pred['result'])
    print("Predicted Grade: " + grade['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: What is the approximate weight of a pair of Women's Campside Oxfords?
Real Answer: The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz.
Predicted Answer: The approximate weight of a pair of Women's Campside Oxfords is 1 lb. 1 oz.
Predicted Grade: CORRECT

Example 3:
Question: What are some of the key features of the Recycled Waterhog Dog Mat, Chevron Weave?
Real Answer: Some key features of the Recycled Waterhog Dog Mat, Chevron Weave include being constructed from recycled plastic materials

In [29]:
graded_outputs[0]

{'results': 'CORRECT'}

## LangChain evaluation platform

The LangChain evaluation platform, LangChain Plus, can be accessed here https://www.langchain.plus/.  
Use the invite code `lang_learners_2023`

Reminder: Download your notebook to you local computer to save your work.